In [99]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

Prvi recommender: content - based

In [100]:
movies = pd.read_csv('https://raw.githubusercontent.com/marinaborozan/RecommenderSystem/main/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [101]:
ratings = pd.read_csv('https://raw.githubusercontent.com/marinaborozan/RecommenderSystem/main/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [102]:
#TMDB dataset sa overview i keywords podacima za svaki film 
columns = ['id','keywords','original_title','overview','tagline','title','vote_average','vote_count']
tmdb_movies = pd.read_csv('https://raw.githubusercontent.com/marinaborozan/RecommenderSystem/main/tmdb_5000_movies.csv', usecols = columns)
tmdb_movies.head()

,id,keywords,original_title,overview,tagline,title,vote_average,vote_count
0,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,Avatar,7.2,11800
1,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,Spectre,6.3,4466
3,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,Following the death of District Attorney Harve...,The Legend Ends,The Dark Knight Rises,7.6,9106
4,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",John Carter,6.1,2124


In [103]:
tmdb_movies.shape

(4803, 8)

In [104]:
#samo kolone koje koristim
content_df_tmdb = tmdb_movies[['id', 'title', 'keywords', 'overview']]
content_df_tmdb.head()

,id,title,keywords,overview
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [105]:
#funkcija za dobijanje keyworda iz kolone
import ast
def extract_keywords(keywords_string):
  keywords_list = ast.literal_eval(keywords_string)
  result = ""
  for dictionary in keywords_list:
    result += dictionary['name']
    result += " " 
  return result[:-1]

In [106]:
#provera kako radi
extract_keywords(content_df_tmdb.iloc[0].keywords)

'culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d'

In [107]:
content_df_tmdb['word_document'] = content_df_tmdb['keywords'].apply(lambda x: extract_keywords(x)) + content_df_tmdb['overview']
content_df_tmdb.drop(columns=['keywords','overview'], inplace=True)
content_df_tmdb.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,title,word_document
0,19995,Avatar,culture clash future space war space colony so...
1,285,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...
2,206647,Spectre,spy based on novel secret agent sequel mi6 bri...
3,49026,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...
4,49529,John Carter,based on novel mars medallion space travel pri...


In [108]:
content_df_tmdb.reset_index(inplace=True)
content_df_tmdb.head()

,index,id,title,word_document
0,0,19995,Avatar,culture clash future space war space colony so...
1,1,285,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...
2,2,206647,Spectre,spy based on novel secret agent sequel mi6 bri...
3,3,49026,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...
4,4,49529,John Carter,based on novel mars medallion space travel pri...


In [109]:
print(content_df_tmdb.id.unique().shape[0])
print(content_df_tmdb.title.unique().shape[0])

4803
4800


In [110]:
#prvi parametar dimenzije je broj redova u datasetu, drugi parametar je broj jedinstvenih reci
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(content_df_tmdb['word_document'].values.astype('U')) 
tfidf_matrix.shape

(4803, 26312)

In [111]:
#ovo ce biti matrica po indexima iz content df
cosine_sim = cosine_similarity(tfidf_matrix)
print(cosine_sim.shape) 

(4803, 4803)


In [112]:
content_df_tmdb = content_df_tmdb.reset_index()
titles = content_df_tmdb['title']
indices_by_title = pd.Series(content_df_tmdb.index,index = content_df_tmdb['title'])

In [113]:
indices_by_title['John Carter']

4

In [114]:
def get_top_15_similar(title, cosine_sim):
    index = indices_by_title[title] 
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True) 
    sim_scores = sim_scores[1:16]
    movie_indices = [score[0] for score in sim_scores]
    return pd.DataFrame(titles.iloc[movie_indices])

In [115]:
get_top_15_similar('Spectre', cosine_sim)

,title
1343,Never Say Never Again
3285,Restless
29,Skyfall
4339,Dr. No
3143,You Only Live Twice
4071,From Russia with Love
147,Die Another Day
3162,Thunderball
11,Quantum of Solace
3343,Live and Let Die


In [116]:
get_top_15_similar('The Dark Knight Rises', cosine_sim)

,title
1359,Batman
428,Batman Returns
65,The Dark Knight
299,Batman Forever
119,Batman Begins
210,Batman & Robin
9,Batman v Superman: Dawn of Justice
2507,Slow Burn
3819,Defendor
879,Law Abiding Citizen


Prvi recommender - ishod: content - based recommender koji gleda slicnost opisa i kljucnih reci za film preporucuje vrlo slicne filmove po sadrzaju i zanru

Drugi recommender: collaborative, item-based, prema cosine_similarity matrici izracunatoj u prvom recommenderu i Pirsonovom koeficijentu - http://cs229.stanford.edu/proj2018/report/128.pdf

*pitanje: ovde pise da se preko Pirs. koef. predvidja rejting ocene za neki film za content-based, ali posto se koristi matrica usera i itema, zar to nije collaborative, i to item-item?

In [117]:
links = pd.read_csv('https://raw.githubusercontent.com/marinaborozan/RecommenderSystem/main/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [118]:
print(links.shape)
links.dropna(inplace=True)
print(links.shape)

(9742, 3)
(9734, 3)


In [119]:
links.tmdbId.isnull().sum()

0

In [120]:
links.tmdbId = links.tmdbId.astype(int)
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


In [121]:
ratings_tmdb = pd.merge(ratings, links, on = 'movieId')
#ratings_tmdb.tmdbId = ratings_tmdb.tmdbId.astype(int)
ratings_tmdb.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,1,4.0,964982703,114709,862
1,5,1,4.0,847434962,114709,862
2,7,1,4.5,1106635946,114709,862
3,15,1,2.5,1510577970,114709,862
4,17,1,4.5,1305696483,114709,862


In [122]:
ratings_final = ratings_tmdb[ratings_tmdb['tmdbId'].isin(content_df_tmdb['id'])].dropna()
ratings_final.tmdbId.nunique()

3535

In [123]:
ratings_final.tmdbId.isnull().sum()

0

In [124]:
n_users = ratings_final.userId.unique().shape[0]
#tmdb id koristim jer posle preko njega trazim po istom tmdb id iz content df-a
#max da se ne resetuju tmdb id-evi jer se u ratingsima ponavljaju
n_items = max(ratings_final.tmdbId)
print ('Number of users: ', n_users, ", number of items: ", n_items)

Number of users:  610 , number of items:  376659


In [125]:
#matrica za racunanje slicnosti izmedju usera i itema
data_matrix = np.zeros((n_users, n_items))
for line in ratings_final.itertuples():
    data_matrix[line[1]-1, line[6]-1] = line[3]

In [126]:
indices_by_tmdb_id = pd.Series(content_df_tmdb.index,index = content_df_tmdb['id'])

In [127]:
def predict_rating(data_matrix, cosine_sim, user_id, title):
    #provera da li je neki naslov koji nema u content df
    if title not in indices_by_title: 
        print('movie with title: ' + title + ' is not found in content dataset')
        return 0
    i = indices_by_title[title]
    rating = 0
    similarity_sum = 0
    #vraca listu tmdbId za filmove koje je user ocenio jer uzmem samo one koji nisu ocena nula - nonzero vraca indexe
    rated_movies_for_user = data_matrix[user_id-1].nonzero()[0]
    #srednja vrednost ocena za usera 
    #pitanje - da li uzeti samo ne-nula u obzir?
    mean = data_matrix[user_id-1, :].mean()
    
    for rated_movie_id in rated_movies_for_user:
      #za sad ovako, srediti zajedno sa ciscenjem datasetova
      if (rated_movie_id+1 not in indices_by_tmdb_id):
        continue
      j = indices_by_tmdb_id[rated_movie_id+1]
      rating += cosine_sim[i][j] * (data_matrix[user_id-1,rated_movie_id] - mean)
      similarity_sum += cosine_sim[i][j]

    if (similarity_sum != 0):
      rating /= similarity_sum
    rating += mean
    return rating

In [128]:
#ovo su userId i tmdbId prosledjeni
predict_rating(data_matrix,cosine_sim,7,'Skyfall')

3.285731601237582

TODO: dodati validaciju da bi videla kako predvidja

Treci recommender: collaborative, SVD iz biblioteke surprise

In [129]:
# installing surprise library
!pip install surprise

In [130]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate

In [131]:
ratings_final_filter = ratings_final[['userId','tmdbId','rating']]
ratings_final_filter.head()

,userId,tmdbId,rating
0,1,862,4.0
1,5,862,4.0
2,7,862,4.5
3,15,862,2.5
4,17,862,4.5


In [132]:
reader = Reader()
data = Dataset.load_from_df(ratings_final_filter, reader)

In [133]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'mae'], cv = 3)

{'fit_time': (2.618328332901001, 2.6553003787994385, 2.621880292892456),
 'test_mae': array([0.6710664 , 0.65794417, 0.6658366 ]),
 'test_rmse': array([0.87447554, 0.85950393, 0.8688091 ]),
 'test_time': (0.1729748249053955, 0.28830933570861816, 0.2373192310333252)}

In [134]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [135]:
#par filmova koje je ocenio user 5 da bih poredila sa onim koje je predvideo svd
ratings_final_filter[ratings_final_filter['userId'] == 5].head()

,userId,tmdbId,rating
1,5,862,4.0
573,5,629,4.0
855,5,197,4.0
1821,5,680,5.0
2317,5,9331,3.0


In [136]:
print(svd.predict(5,862))
print(svd.predict(5,629))
print(svd.predict(5,197))
print(svd.predict(5,680))
print(svd.predict(5,9331))

user: 5          item: 862        r_ui = None   est = 3.79   {'was_impossible': False}
user: 5          item: 629        r_ui = None   est = 4.23   {'was_impossible': False}
user: 5          item: 197        r_ui = None   est = 3.94   {'was_impossible': False}
user: 5          item: 680        r_ui = None   est = 4.26   {'was_impossible': False}
user: 5          item: 9331       r_ui = None   est = 3.29   {'was_impossible': False}


In [137]:
trainset, testset = train_test_split(data, test_size=.25)
algorithm = SVD()
algorithm.fit(trainset)
predictions = algorithm.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8659


0.8659086676336254

In [138]:
tmdb_id_titles_dict = dict(zip(content_df_tmdb.id, content_df_tmdb.title))
#tmdb_id_titles_dict

In [139]:
def get_top_10_svd(user_id):
  estimated = []
  for tmdb_id in ratings_final.tmdbId.unique():
    estimate = svd.predict(user_id, tmdb_id).est
    estimated.append((estimate,tmdb_id_titles_dict[tmdb_id]))

  sorted_estimated = sorted(estimated, key=lambda tup: tup[0], reverse=True)
  #sorted_estimated =  sorted(estimated, reverse=True)
  return pd.DataFrame(sorted_estimated[1:11], columns=['rating','title'])

In [140]:
get_top_10_svd(13)

,rating,title
0,4.733464,Pulp Fiction
1,4.711249,Memento
2,4.676909,The Matrix
3,4.670897,The Silence of the Lambs
4,4.629077,Star Wars
5,4.603147,Donnie Darko
6,4.579081,Billy Elliot
7,4.572185,Brazil
8,4.563815,Dr. Strangelove or: How I Learned to Stop Worr...
9,4.562050,Fight Club


In [141]:
get_top_10_svd(23)

,rating,title
0,4.016643,Das Boot
1,4.010401,Blade Runner
2,3.903826,Rosemary's Baby
3,3.899749,Patton
4,3.888418,The Great Escape
5,3.885602,Trainspotting
6,3.883966,The Green Mile
7,3.883699,Spotlight
8,3.875797,Eternal Sunshine of the Spotless Mind
9,3.863789,Spirited Away


Treci recommender - ishod: za dva razlicita usera se dobiju razlicite preporuke za top 10 filmova, samo na osnovu svd predvidjanja za matricu usera i itema

RMSE: 0.8619

Hibridni recommender: prvi pokusaj hibridnog modela - cosine_sim matrica + svd

In [142]:
def predict_hybrid(user_id, movie_title):
    index = indices_by_title[movie_title] 
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True) 
    sim_scores = sim_scores[1:101]
    movie_indices = [score[0] for score in sim_scores]
    movie_data_for_indices = content_df_tmdb.iloc[movie_indices]
    movie_data_for_indices['est'] = movie_data_for_indices['id'].apply(lambda x: svd.predict(user_id, id).est)
    movie_data_for_indices = movie_data_for_indices.sort_values('est', ascending=False)
    return movie_data_for_indices.head(10)

In [143]:
predict_hybrid(5, 'Toy Story')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,level_0,index,id,title,word_document,est
343,343,343,863,Toy Story 2,museum prosecution identity crisis airplane fl...,3.366144
889,889,889,8741,The Thin Red Line,based on novel japanese world war ii battle as...,3.366144
4138,4138,4138,115210,Stitches,slapstick teen angst slaughter dark comedy sup...,3.366144
915,915,915,9441,Stepmom,divorce rebellious daughter freeze frame custo...,3.366144
1494,1494,1494,10719,Elf,holiday elves santa claus looking for birth p...,3.366144
1519,1519,1519,3587,Bogus,circus magic aunt imaginary friendRecently orp...,3.366144
3508,3508,3508,37206,A Shine of Rainbows,orphan rainbowAn orphaned boy named Tomás is a...,3.366144
197,197,197,671,Harry Potter and the Philosopher's Stone,witch christmas party magic cutting the cord h...,3.366144
837,837,837,175574,Free Birds,holiday thanksgiving freedom duringcreditsstin...,3.366144
2082,2082,2082,50780,The Beaver,puppet woman directorSuffering from a severe c...,3.366144


In [144]:
predict_hybrid(7, 'Toy Story')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,level_0,index,id,title,word_document,est
343,343,343,863,Toy Story 2,museum prosecution identity crisis airplane fl...,3.036044
889,889,889,8741,The Thin Red Line,based on novel japanese world war ii battle as...,3.036044
4138,4138,4138,115210,Stitches,slapstick teen angst slaughter dark comedy sup...,3.036044
915,915,915,9441,Stepmom,divorce rebellious daughter freeze frame custo...,3.036044
1494,1494,1494,10719,Elf,holiday elves santa claus looking for birth p...,3.036044
1519,1519,1519,3587,Bogus,circus magic aunt imaginary friendRecently orp...,3.036044
3508,3508,3508,37206,A Shine of Rainbows,orphan rainbowAn orphaned boy named Tomás is a...,3.036044
197,197,197,671,Harry Potter and the Philosopher's Stone,witch christmas party magic cutting the cord h...,3.036044
837,837,837,175574,Free Birds,holiday thanksgiving freedom duringcreditsstin...,3.036044
2082,2082,2082,50780,The Beaver,puppet woman directorSuffering from a severe c...,3.036044


TODO: Dodati preporuku po zanrovima, kao i popularnosti, za bolje rezultate u hibridnom modelu